<h4 align="left">Contents</h4>



In [ ]:
from sympy import Function, dsolve, Derivative, symbols
import numpy as np

class ODE:

    def __init__(self, order: int, coef: list[float], func_str: str | None = None):
        """
        # Initialize an ODE object (order - int: order of the ode, coef - list[float]: coefficients, func_str - str: string representaiton of the ode)
        
        # Error checking: ValueError - If order is negative or if coefficients list is invalid
        """ 
        # Validate order
        if not isinstance(order, int) or order < 0:
            raise ValueError("Order must be a non-negative integer")
        self.order = order
        
        # Validate coefficients
        if not isinstance(coef, (list, np.ndarray)) or len(coef) != order + 1:
            raise ValueError(f"Coefficients must be a list of length {order + 1}")
        self.coef = list(coef)
        
        # Initialize symbolic variables
        self.x, self.y = symbols('x y')
        self.f = Function('y')(self.x)
        
        # Create the ODE expression
        self.equation = self._create_equation()
        
        # Store the function string if provided
        self.func_str = func_str if func_str else str(self.equation)

    def _create_equation(self):
        """
        Creates the symbolic representation of the ODE.
        
        Returns: sympy expression - The left-hand side of the ODE
        """
        equation = 0
        
        # Add terms for each derivative
        for i in range(self.order + 1):
            if self.coef[i] != 0:
                if i == 0:
                    equation += self.coef[i] * self.f
                else:
                    equation += self.coef[i] * Derivative(self.f, self.x, i)
        
        return equation

    def _get_derivative_notation(self, order: int) -> str:
        """
        Returns the appropriate derivative notation based on order.
        
        Args: order - The order of the derivative
            
        Returns: str -  The derivative notation
        """
        if order == 0:
            return "y"
        elif order == 1:
            return "y'"
        elif order == 2:
            return "y''"
        else:
            return f"y^({order})"

    def __str__(self) -> str:
        """
        Returns a readable string representation of the ODE.
        
        Returns: str - String representation of the ODE
        """
        terms = []
        
        for i in range(self.order, -1, -1):
            if self.coef[i] != 0:
                # Handle coefficient
                coef_str = ''
                if self.coef[i] == 1 and i > 0:  # Don't show coefficient 1 except for y term
                    coef_str = ''
                elif self.coef[i] == -1 and i > 0:
                    coef_str = '-'
                else:
                    coef_str = f"{self.coef[i]}"
                    if self.coef[i] > 0 and terms:  # Add + for positive terms after first
                        coef_str = f"+ {coef_str}"
                    elif self.coef[i] < 0:  # Handle negative terms
                        coef_str = f"- {abs(self.coef[i])}"
                
                # Get derivative notation
                term = f"{coef_str}{self._get_derivative_notation(i)}"
                terms.append(term)
        
        equation_str = ' '.join(terms) if terms else "0"
        return f"ODE: {equation_str} = 0"
    
    def solve(self, ics: dict[tuple[str, int], float] | None = None) -> str:
        """
        Solves the ODE using SymPy's dsolve.
        
        Args: ics (dict, optional) - Initial conditions in the form {(y, 0): value, (y, 1): value, ...}
        
        Returns: str - Solution to the ODE
            
        Raises: ValueError - If the equation cannot be solved
        """
        try:
            if ics:
                solution = dsolve(self.equation, self.f, ics=ics)
            else:
                solution = dsolve(self.equation, self.f)
            return str(solution)
        except Exception as e:
            raise ValueError(f"Could not solve ODE: {str(e)}")
    
    @property
    def characteristic_equation(self) -> list[float]:
        """
        Returns the coefficients of the characteristic equation for linear ODEs with constant coefficients.
        
        Returns: list[float]: Coefficients of the characteristic equation
        """
        return self.coef[::-1]  # Reverse the coefficients

In [15]:
ode5 = ODE(order=5, coef=[-1, 1, -2, 1, -2, 1])
print(ode5)
solution = ode5.solve()
print(f"Solution: {solution}")

ODE: y^(5) - 2y^(4) y^(3) - 2y'' y' - 1y = 0
Solution: Eq(y(x), C5*exp(x*CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 0)) + (C1*sin(x*im(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 1))) + C2*cos(x*im(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 1))))*exp(x*re(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 1))) + (C3*sin(x*im(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 3))) + C4*cos(x*im(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 3))))*exp(x*re(CRootOf(_x**5 - 2*_x**4 + _x**3 - 2*_x**2 + _x - 1, 3))))
